# ANALYSIS OF CAR MODELS 🛰 🚀 🛸 🚁

![](https://p4.wallpaperbetter.com/wallpaper/176/276/899/lamborghini-wallpaper-preview.jpg)

### Import Libraries and Dataset 🦚

- **Libraries**

In [5]:
# Main Packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# In case we want to ignore the warning parts @-@
import warnings
warnings.filterwarnings("ignore")

- **Dataset**

In [7]:
# See all csv files in the folder 🐯
import os
files = [f for f in os.listdir() if f.endswith(".csv")]
print(files)

['cleaned_car_buyers.csv', 'Engine CC.csv', 'Female.csv', 'Fuel.csv', 'Male.csv', 'Manufacturer.csv', 'Model.csv', 'Power.csv', 'Price.csv', 'Total.csv', 'Transmission.csv', 'Unknown.csv']


In [8]:
data= pd.read_csv("/kaggle/input/car-buyers/CarBuyers.csv")
# Sample of data
data.head(3)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/car-buyers/CarBuyers.csv'

- Just a curiousity, why sum of num of male, female and unknown does not equal total number of owners, where the remaining owners are classified into ^^.

### Data Inspection and Pre-Processing 🦌

In [ ]:
data.shape

In [ ]:
data.columns

- **Data Format**
- Categorical variables: 'Manufacturer', 'Model','Fuel'
- Numeric variables: 
    - Continuous: 'Price', 'Transmission', 'Power', 'Engine CC'
    - Discrete: 'Male', 'Female', 'Unknown', 'Total'

In [ ]:
data.info()

In [ ]:
# We convert number of Male, female, unknown and total ownwers to integer 🐐
def to_int(x):
    return(int(x.replace(',','')))
for int_col in ['Male', 'Female', 'Unknown', 'Total']:
    data[int_col]= data[int_col].map(to_int)
data.info()

- There is no missing values in all attributes 

In [ ]:
# Unique values of each categorical variable 🐩
unique_data= pd.DataFrame({'column':  ['Manufacturer', 'Model','Fuel']})
unique_data['is_unique']= unique_data['column'].apply(lambda x: data[x].is_unique)
unique_data['num unique val']= unique_data['column'].apply(lambda x: data[x].nunique())
unique_data['unique val']= unique_data['column'].apply(lambda x: data[x].unique())
unique_data

- There is no single attribute that can uniquely identify each record.  

In [ ]:
# Some descriptive statistic of numeric vaiables 🦝
data.describe().round(2)

### Basic EDA and Data Pre-processing 🐬 🐳 🐋

- I observe that there are many records with the same 'Manufacturer', 'Model', 'Price', 'Transmission', 'Power', 'Engine CC','Fuel' values, with different number of owners. I'll show you 

In [ ]:
data.groupby(['Manufacturer', 'Model', 'Price', 'Transmission', 'Power', 'Engine CC','Fuel'],
             as_index=False)['Total'].count().head(5)

In [ ]:
# U can check others as well
mask1= data['Manufacturer']=='Abarth'
mask2= data['Model']=='500C'
data[mask1&mask2]

- One way to deal with this is to sum up all number of owners of the same type of record

In [ ]:
data= data.groupby(['Manufacturer', 'Model', 'Price', 'Transmission', 'Power', 'Engine CC','Fuel'],
             as_index=False).sum()
data.head(3)

- Now, the composite key of Manufacturer and Model can uniquely identify a record. Lets check!

In [ ]:
data.set_index(['Manufacturer','Model']).index.is_unique

**Car Owners by Gender** 🍄

In [ ]:
data_gender= pd.DataFrame({'Gender':['Male', 'Female', 'Unknown']})
data_gender['Count']= data_gender['Gender'].apply(lambda x: data[x].sum())
data_gender['%']= data_gender['Count'].apply(lambda x: x/data_gender['Count'].sum())
data_gender

In [ ]:
fig,ax=plt.subplots(ncols=2,figsize=(12,5))
sns.barplot(x='Gender', y='Count', data=data_gender, palette='rocket',edgecolor='black', ax=ax[0])
sns.barplot(x='Gender', y='%', data=data_gender, palette='viridis',edgecolor='black', ax=ax[1])
ax[0].set_title('Number of Car Owners by Gender', color='darkred', fontsize=15)
ax[1].set_title('Percentage of Car Owners by Gender', color='darkblue', fontsize=15)
fig.tight_layout();

- A majority of Car owners are male (with approximately 60%), amount of female owners is about two third, number of owners with unidentified-gender is of lowest with just under 5%. 

**Top Manufacturers by Number of Owners** 🍁

- I want to identify top manufacturers by amount of buyers, whether there is any dominated Manufacturers or not. 

In [ ]:
top_manufacturer= data.groupby('Manufacturer', as_index=False)['Male', 'Female','Total'].sum()
top_manufacturer.head()

In [ ]:
fig, ax= plt.subplots(figsize=(15,6))
sns.barplot(x='Manufacturer', y='Total',data=top_manufacturer.sort_values('Total',
            ascending=False).head(20), palette='flare',edgecolor='black', ax=ax)
ax.tick_params(axis='x', labelrotation=45)
ax.set_title('Top 20 Manufacturers by Total Number of Owners', fontsize=15);

In [ ]:
fig,ax= plt.subplots(ncols=2, figsize=(15,5))
sns.barplot(x='Manufacturer', y='Male',data=top_manufacturer.sort_values('Male',ascending=False).head(8),
           ax=ax[0], palette='viridis',edgecolor='black')
sns.barplot(x='Manufacturer', y='Female',data=top_manufacturer.sort_values('Female',ascending=False).head(8),
           ax=ax[1], palette='mako',edgecolor='black')
ax[0].set_title('Top 8 Manufacturers by Total Number of Male Owners', fontsize=15)
ax[1].set_title('Top 8 Manufacturers by Total Number of Female Owners', fontsize=15)
fig.tight_layout();

- In general, Ford is the one that dominates all charts in term of purchasers, the Top Manufacturers can be listed are Ford, Peugeot, Volkswagen, Renault. There is no significant differences in the choice of Manufacturers between male and female customers. 

**Top Models by number of owners** 🎍

- In similar spirit, we find top Models in term of number of owners.

In [ ]:
top_model= data.groupby(['Model','Manufacturer'], as_index=False)['Male', 'Female','Total'].sum()
top_model.head()

In [ ]:
fig, ax= plt.subplots(figsize=(15,6))
sns.barplot(x='Model', y='Total',data=top_model.sort_values('Total',
            ascending=False).head(20), palette='flare', ax=ax,edgecolor='black')
ax.tick_params(axis='x', labelrotation=45)
ax.set_title('Top 20 Models by Total Number of Owners', fontsize=15);

In [ ]:
fig,ax= plt.subplots(ncols=2, figsize=(15,5))
sns.barplot(x='Model', y='Male',data=top_model.sort_values('Male',ascending=False).head(8),
           ax=ax[0], palette='viridis',edgecolor='black')
sns.barplot(x='Model', y='Female',data=top_model.sort_values('Female',ascending=False).head(8),
           ax=ax[1], palette='mako',edgecolor='black')
ax[0].set_title('Top 8 Models by Total Number of Male Owners', fontsize=15)
ax[1].set_title('Top 8 Models by Total Number of Female Owners', fontsize=15)
fig.tight_layout();

- All in all, in term of total number of owners, male or female owners, Fiesta is always placed at first position. 
- Interestingly, there is a notable differences in the choice of Male and Female buyers. For male buyers, Fiesta and Escort outnumbered all other models, however the gap is not significant. On the other hand, Fiesta dominates the figures of Female, the second top model (Micra) doesnt have even half of Fiesta amount. 
- In addition, except Fiesta,Golf and Focus, others appear in top preferrence for male, dont appear in Female chart. 

In [ ]:
# We can see the Top Car models of each manufacturer, 
# and see how the results conincide with top ranked list of manufacturers 🔮
top_model_all=top_model.sort_values('Total',ascending=False).head(20).pivot_table(index='Manufacturer',
                 margins=True,aggfunc= np.sum,columns='Model',values='Total').sort_values('All',ascending=False)
fig,ax=plt.subplots(figsize=(15,7))
top_model_all.iloc[1:,:-1].plot(kind='bar', stacked='True',ax=ax, cmap='viridis', edgecolor='black')
ax.set_title('Manufacturers by Top 20 Car Models (Total Owners)', fontsize=15);

- We can see that most of the Manufacturers ranked on top number of owners have top models. Obvious one is Ford with up to 6 models ranked on top (One of them is the 'famous' Fiesta).
- However, there are some manufacturers listed on top 10 based on number of owners which does not have car model ranked on top 20. Lets check !!!

In [ ]:
top_model_manu_all= top_model_all[1:].index.tolist()
top_manu_all= top_manufacturer.sort_values('Total',ascending=False).head(10)['Manufacturer'].tolist()
intersection= list(set(top_model_manu_all)&set(top_manu_all))
print('Manufacturers appear in both chart:\n',intersection)
print('Top Manufacturers with no Models on top 20:\n', set(top_manu_all)-set(intersection))

- Similarly, we'll have a look at Manufacturers by Top ranked models by Male and Female Owners. 

In [ ]:
# Male Owners
top_model_male= top_model.sort_values('Male',ascending=False).head(10).pivot_table(index='Manufacturer',
                 margins=True,aggfunc= np.sum,columns='Model',values='Male').sort_values('All',ascending=False)
# Female Owners
top_model_female= top_model.sort_values('Female',ascending=False).head(10).pivot_table(index='Manufacturer',
                 margins=True,aggfunc= np.sum,columns='Model',values='Female').sort_values('All',ascending=False)
# Viz Time !
fig, ax= plt.subplots(ncols=2, figsize=(15,6))
top_model_male.iloc[1:,:-1].plot(kind='bar',stacked=True, cmap='icefire',ax=ax[0], edgecolor='black')
top_model_female.iloc[1:,:-1].plot(kind='bar', stacked=True, cmap='magma', ax=ax[1], edgecolor='black')
ax[0].set_title('Manufacturers by Top 10 Car Models (Male Owners)', fontsize=15)
ax[1].set_title('Manufacturers by Top 10 Car Models (Female Owners)', fontsize=15)
fig.tight_layout();

**A few insights of Fuel attribute** 🧬

In [ ]:
fuel_data= data.groupby('Fuel', as_index=False)['Model'].count()
fig, ax= plt.subplots(figsize=(7,5))
sns.barplot(x='Fuel', y='Model', data=fuel_data, palette='rocket', edgecolor='black', ax=ax)
ax.set_title('Number of Models by type of Fuel', fontsize=12)
for i,v in enumerate(fuel_data['Model'].tolist()):
    ax.text(x=i, y=v+2, s=f"{v}" , fontdict=dict(fontsize=12))

- Clearly, most of the car models use Petrol as their fuel (up to 85%), automatic fuel and diesel are accounted for humble amount , may be because it is too expensive @-@. Lets check !

In [ ]:
fig, ax= plt.subplots(figsize=(7,5))
df= data.groupby('Fuel',as_index=False)['Price'].mean()
sns.barplot(x='Fuel', y='Price', data= df
            , palette='mako', edgecolor='black', ax=ax)
ax.set_title('Average Price of Cars by each type of Fuel', fontsize=12)
for i,v in enumerate(df['Price'].tolist()):
    ax.text(x=i, y=v, s=f"{np.round(v,2)}" , fontdict=dict(fontsize=12))

- Surprisingly, There is no significant gap between the Average price for Cars using Petrol and other 2, so Price is not a decisive factor determining why so many car models using this type of fuel. 

**Histogram, Correlation Matrix Plot and Pair Plot for Numeric variables** 🐇

- I aim to use some Regression models to predict the Price of Cars,hence, there is a need to explore the relationship between 'potential' predictors ('Transmission', 'Power', 'Engine CC') and Price variable (our target)

In [ ]:
predictor= ['Transmission', 'Power', 'Engine CC']
target= 'Price'

In [ ]:
# Histogram of all Variables
fig, ax= plt.subplots(nrows= 2, ncols=2, figsize=(15,8))
ax_index=[ax[0,0],ax[0,1],ax[1,0]]
ax[1,1].hist(data[target].clip(upper=350),bins=40,color='maroon', edgecolor='black')
ax[1,1].set_title('Histogram of Car Price (Target)', color='darkred')
for i in range(3):
    ax_index[i].hist(data[predictor[i]],bins=40,color='navy', edgecolor='black')
    ax_index[i].set_title(f'Histogram of {predictor[i]} (Predictor)')
fig.tight_layout();

- Actually, There is not much things interpretted from these histogram. The Price (target) is extremely right-skewed (so we may expect there are some outliers here), while the level of skewness is lower in the predictors. 
- All variables seem to follow bell-shaped distribution, except for transmission which has unclear distribution. Nonetheless, these are not big problems, lets see relationship amongs variables !

In [ ]:
sns.pairplot(data=data[predictor+[target]], kind='reg', diag_kws={'bins':20, 'color':'darkblue','edgecolor':'black'},
            plot_kws={'marker':'.','color':'darkred'}, size=1.8);

- By the scatter plot of predictor variables against Price, we can see strong relationship 
- We can observe that there are a few data points (2-6 points) that the price is extremely high, which may affect the performance of Regression model, we have to handle these points ! 
- I use Capping to deal with these points, in other words, all price record over 99% quantitle (5 points)'ll be replaced by 99% quantile value. Let see the result. 

In [ ]:
data2= data.copy()
data2['Price']= data2['Price'].clip(upper=data['Price'].quantile(0.99))
sns.pairplot(data=data2[predictor+[target]], kind='reg', diag_kws={'bins':20, 'color':'darkblue','edgecolor':'black'},
            plot_kws={'marker':'.','color':'darkred'}, size=2);

- Now, they look much better, Lets see how variables are correlated with each other. 

In [ ]:
# Correlation Matrix plot 🎎
fig, ax= plt.subplots(figsize=(8,6))
sns.heatmap(data2[predictor+[target]].corr(), annot=True, cmap='viridis',
            fmt='0.3f',ax=ax)
ax.set_title('Correlation Matrix Plot', fontsize=15);

- Interestingly, the Price is positively highly-correlated with Engine CC (0.828) and Power (0.942), while the relationship seems unsignificant btw Price and Transmission (0.348)
- Additionally, we see high correlation btw Power and Engine CC, which may cause Multi-Collinearity in our regression model. We can use VIF metric to check our assumption. 

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
X=data2[predictor]
vif_info= pd.DataFrame()
vif_info['Column']= X.columns
vif_info['VIF']=[variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info

- High value of VIF of Power and Engine CC indicates that they are highly collinear with other variables in the model.
- Simplest way (but usually not recommended) is to drop the one with highest value of VIF, which is Engine CC in this case. 

In [ ]:
# Lets see VIF measures when we drop Engine CC variable 🚨 
X=data2[['Transmission','Power']]
vif_info= pd.DataFrame()
vif_info['Column']= X.columns
vif_info['VIF']=[variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info

- Now, the value of VIF is much lower (less than 5), so we can be certain that there is no multilinearity that can affect the result of our regression models. 

## Trainning Regression Models 🚀

### Simple Linear Regression Model 🔫

- We use only Power as the predictor for Price

**Independent and Dependent variables** 🚦 🚥

In [ ]:
X= data2['Power'].values
y= data2['Price'].values
X= X.reshape(-1,1)
y=y.reshape(-1,1)
display(X[:3], y[:3])

**Train-Test Split** ⚖️

In [ ]:
# Split the data into train (80%) and test (20%) set 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=0)

**Training the model on Train set** 🛒

In [ ]:
from sklearn.linear_model import LinearRegression
regressor= LinearRegression()
regressor.fit(X_train, y_train) 

In [ ]:
# Model coefficients 
display(regressor.coef_, regressor.intercept_)

**Model Performance** 📚

In [ ]:
# Prediction on the train and test set 
train_diff= y_train- regressor.predict(X_train)
test_diff = y_test - regressor.predict(X_test)
fig,ax = plt.subplots(ncols=2, figsize=(10,4))
ax[0].hist(train_diff, bins=40, color='lightcoral', edgecolor='black', density =True)
ax[1].hist(test_diff, bins=40, color='bisque', edgecolor='black', density =True)
ax[0].set_title('Residual histogram (Train set)')
ax[1].set_title('Residual histogram (Test set)')
fig.tight_layout();

In [ ]:
print('Mean of Residuals:',[np.mean(train_diff), np.mean(test_diff)])
print('Var of Residuals:',[np.var(train_diff), np.var(test_diff)])

In [ ]:
# R2 measure
from sklearn.metrics import r2_score
display(r2_score(y_train,regressor.predict(X_train)),r2_score(y_test,regressor.predict(X_test)))

- The model perform quite well with no significant difference in performance of train and test set. 

**Model Visualization** 🧮

In [ ]:
fig,ax= plt.subplots(ncols=2, figsize=(13,5))
# Train set
ax[0].scatter(X_train, y_train, color = 'darkblue', marker='*', s=70)
ax[0].plot(X_train, regressor.predict(X_train), color = 'black')
ax[0].set_title('Simple Regression model (Train set)', color='darkblue', fontsize=15)
# Test set
ax[1].scatter(X_test, y_test, color = 'darkred', marker='^', s=70)
ax[1].plot(X_train, regressor.predict(X_train), color = 'black')
ax[1].set_title('Simple Regression model (Test set)', color='darkred', fontsize=15)
fig.tight_layout()

### Multiple Linear Regression Model 🦠

- We use both Transmission and Power as predictors in this model

**Independent and Dependent variables** 🚦 🚥

In [ ]:
X= data2[['Transmission', 'Power']].values
y= data2['Price'].values
display(X[:3], y[:3])

**Train-Test Split** ⚖️

In [ ]:
# Split the data into train (80%) and test (20%) set 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=0)

**Training the model on Train set** 🛒

In [ ]:
from sklearn.linear_model import LinearRegression
regressor= LinearRegression()
regressor.fit(X_train, y_train) 

In [ ]:
# Model coefficients 
display(regressor.coef_, regressor.intercept_)

**Model Performance** 📚

In [ ]:
# Prediction on the train and test set 
train_diff= y_train- regressor.predict(X_train)
test_diff = y_test - regressor.predict(X_test)
fig,ax = plt.subplots(ncols=2, figsize=(10,4))
ax[0].hist(train_diff, bins=40, color='darkmagenta', edgecolor='black', density =True)
ax[1].hist(test_diff, bins=40, color='darkblue', edgecolor='black', density =True)
ax[0].set_title('Residual histogram (Train set)')
ax[1].set_title('Residual histogram (Test set)')
fig.tight_layout();

In [ ]:
print('Mean of Residuals:',[np.mean(train_diff), np.mean(test_diff)])
print('Var of Residuals:',[np.var(train_diff), np.var(test_diff)])

In [ ]:
# R2 measure
from sklearn.metrics import r2_score
display(r2_score(y_train,regressor.predict(X_train)),r2_score(y_test,regressor.predict(X_test)))

- The model perform quite well with no significant difference in performance of train and test set. 

**Model Visualization** 🧮

In [ ]:
x_surf, y_surf = np.meshgrid(np.linspace(data2['Power'].min(),data2['Power'].max(), 100),
                             np.linspace(data2['Transmission'].min(),data2['Transmission'].max(), 100))
X = pd.DataFrame({'Power': x_surf.ravel(), 'Transmission': y_surf.ravel()})
Y=regressor.predict(X)

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(22,12))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data2['Power'],data2['Transmission'],data2['Price'],c='darkred', marker='o', alpha=0.7)
ax.plot_surface(x_surf,y_surf,Y.reshape(x_surf.shape), color='green', alpha=0.3)
ax.set_xlabel('Power')
ax.set_ylabel('Transmission')
ax.set_zlabel('Price')
ax.set_title('3D plot of Multiple Regression Model', fontsize=15);

### Support Vector Regression Model ⚖️

**Independent and Dependent variables** 🚦 🚥

In [ ]:
X= data2['Power'].values
y= data2['Price'].values
X= X.reshape(-1,1)
y=y.reshape(-1,1)
display(X[:3], y[:3])

**Feature Scaling** 🏮

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

**Train-Test Split** ⚖️

In [ ]:
# Split the data into train (80%) and test (20%) set 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=0)

**Training the model on Train set** 🛒

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

In [ ]:
# Parameters
regressor.get_params()

**Model Performance** 📚

In [ ]:
# Prediction on the train and test set 
train_diff= y_train- regressor.predict(X_train).reshape(-1,1)
test_diff = y_test - regressor.predict(X_test).reshape(-1,1)
fig,ax = plt.subplots(ncols=2, figsize=(10,4))
ax[0].hist(train_diff, bins=40, color='maroon', edgecolor='black', density =True)
ax[1].hist(test_diff, bins=40, color='darkblue', edgecolor='black', density =True)
ax[0].set_title('Residual histogram (Train set)')
ax[1].set_title('Residual histogram (Test set)')
fig.tight_layout();

In [ ]:
print('Mean of Residuals:',[np.mean(train_diff), np.mean(test_diff)])
print('Var of Residuals:',[np.var(train_diff), np.var(test_diff)])

In [ ]:
# R2 measure
from sklearn.metrics import r2_score
display(r2_score(y_train,regressor.predict(X_train).reshape(-1,1)),
        r2_score(y_test,regressor.predict(X_test).reshape(-1,1)))

- The model perform quite well with no significant difference in performance of train and test set. 

**Model Visualization** 🧮

In [ ]:
min_X, max_X= np.min(X_train),np.max(X_train)
X_grid= np.linspace(min_X, max_X, 1000).reshape(-1,1)
fig,ax= plt.subplots(ncols=2, figsize=(13,5))
# Train set
ax[0].scatter(sc_X.inverse_transform(X_train), sc_y.inverse_transform(y_train), color = 'darkblue', marker='*', s=70)
ax[0].scatter(sc_X.inverse_transform(X_grid), sc_y.inverse_transform(regressor.predict(X_grid)), color = 'black',s=5)
ax[0].set_title('Support Vector Regression Model (Train set)', color='darkblue', fontsize=15)
# Test set
ax[1].scatter(sc_X.inverse_transform(X_test), sc_y.inverse_transform(y_test), color = 'darkred', marker='^', s=70)
ax[1].scatter(sc_X.inverse_transform(X_grid), sc_y.inverse_transform(regressor.predict(X_grid)), color = 'black',s=5)
ax[1].set_title('Support Vector Regression Model (Test set)', color='darkred', fontsize=15)
fig.tight_layout()

### Random Forest Regression 🧮

- We use only Power as the predictor for Price

**Independent and Dependent variables** 🚦 🚥

In [ ]:
X= data2['Power'].values
y= data2['Price'].values
X= X.reshape(-1,1)
y=y.reshape(-1,1)
display(X[:3], y[:3])

**Train-Test Split** ⚖️

In [ ]:
# Split the data into train (80%) and test (20%) set 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=0)

**Training the model on Train set** 🛒

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
regressor.fit(X_train, y_train)

In [ ]:
# Parameters
regressor.get_params()

**Model Performance** 📚

In [ ]:
# Prediction on the train and test set 
train_diff= y_train- regressor.predict(X_train).reshape(-1,1)
test_diff = y_test - regressor.predict(X_test).reshape(-1,1)
fig,ax = plt.subplots(ncols=2, figsize=(10,4))
ax[0].hist(train_diff, bins=40, color='lightcoral', edgecolor='black', density =True)
ax[1].hist(test_diff, bins=40, color='bisque', edgecolor='black', density =True)
ax[0].set_title('Residual histogram (Train set)')
ax[1].set_title('Residual histogram (Test set)')
fig.tight_layout();

In [ ]:
print('Mean of Residuals:',[np.mean(train_diff), np.mean(test_diff)])
print('Var of Residuals:',[np.var(train_diff), np.var(test_diff)])

In [ ]:
# R2 measure
from sklearn.metrics import r2_score
display(r2_score(y_train,regressor.predict(X_train)),r2_score(y_test,regressor.predict(X_test)))

- The model seems to over-fit. It perform really well on Train set, but not the case for Test set. 

**Model Visualization** 🧮

In [ ]:
min_X, max_X= np.min(X_train),np.max(X_train)
X_grid= np.linspace(min_X, max_X, 1000).reshape(-1,1)
fig,ax= plt.subplots(ncols=2, figsize=(13,5))
# Train set
ax[0].scatter(X_train, y_train, color = 'darkblue', marker='*', s=70)
ax[0].plot(X_grid, regressor.predict(X_grid), color = 'black')
ax[0].set_title('Random Forest Regression Model (Train set)', color='darkblue', fontsize=15)
# Test set
ax[1].scatter(X_test,y_test, color = 'darkred', marker='^', s=70)
ax[1].plot(X_grid, regressor.predict(X_grid), color = 'black')
ax[1].set_title('Random Forest Regression Model (Test set)', color='darkred', fontsize=15)
fig.tight_layout()